In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pyterrier as pt

if not pt.started():
    pt.init()

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

pd.set_option('display.max_colwidth',200)

C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_13016\1706794743.py:9: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_13016\1706794743.py:10: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()
[nltk_data] Downloading package punkt to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt

In [28]:
lawyers = pd.read_csv(r"Data/lawyers.csv").rename(columns={
    "id":"lawyer_id"
}).drop(columns={"email","email_verified_at","password",
                 "remember_token","phone","affiliation_date","created_at","updated_at"})
lawyers.head()

,lawyer_id,name,address,union_branch,union_number,years_of_experience
0,1,Lawyer_1,"1 Main St, City_1",Branch_1,4211,20
1,2,Lawyer_2,"2 Main St, City_2",Branch_2,4910,14
2,3,Lawyer_3,"3 Main St, City_3",Branch_3,5038,4
3,4,Lawyer_4,"4 Main St, City_4",Branch_4,5715,21
4,5,Lawyer_5,"5 Main St, City_5",Branch_5,3864,18


In [29]:
lawyers['years_of_experience'] = lawyers['years_of_experience'].apply(lambda x: f"{x}year")
lawyers.head()

,lawyer_id,name,address,union_branch,union_number,years_of_experience
0,1,Lawyer_1,"1 Main St, City_1",Branch_1,4211,20year
1,2,Lawyer_2,"2 Main St, City_2",Branch_2,4910,14year
2,3,Lawyer_3,"3 Main St, City_3",Branch_3,5038,4year
3,4,Lawyer_4,"4 Main St, City_4",Branch_4,5715,21year
4,5,Lawyer_5,"5 Main St, City_5",Branch_5,3864,18year


In [30]:
rates = pd.read_csv(r"Data\rates.csv").drop(columns={"created_at","updated_at","id"})
rates.head()

,lawyer_id,user_id,rating,review
0,127,1558,2,Very professional and helpful.
1,141,1157,5,Not satisfied with the service.
2,148,1355,3,Very professional and helpful.
3,189,1050,4,Not satisfied with the service.
4,144,1658,1,"Excellent service, highly recommend!"


In [31]:
rates['rating'] = rates['rating'].apply(lambda x :f"{x}star")
rates.head()

,lawyer_id,user_id,rating,review
0,127,1558,2star,Very professional and helpful.
1,141,1157,5star,Not satisfied with the service.
2,148,1355,3star,Very professional and helpful.
3,189,1050,4star,Not satisfied with the service.
4,144,1658,1star,"Excellent service, highly recommend!"


In [32]:
issues = pd.read_csv(r"Data\issues.csv")
issues.head()

,id,base_number,record_number,lawyer_id,user_id,court_name,type,start_date,end_date,status,estimated_cost,is_active,success_rate,created_at,updated_at
0,1,17432,REC-7578,12,16,Family Court,Criminal,2022-08-23,2025-02-27,Open,1490.15,True,0.11,2022-03-17,2022-09-21
1,2,54708,REC-1121,14,64,District Court,Labor,2024-04-23,2025-08-31,In Progress,2658.10,True,0.62,2023-09-02,2024-10-08
2,3,88493,REC-2156,20,16,Labor Court,Civil,2023-05-30,2025-10-12,Open,4313.33,False,0.67,2023-05-05,2024-10-09
3,4,93489,REC-2439,11,96,Labor Court,Labor,2022-03-19,2025-03-17,In Progress,4565.10,False,0.56,2022-07-23,2024-09-20
4,5,16245,REC-7549,15,74,Labor Court,Commercial,2024-01-21,2025-08-07,Closed,3519.01,False,0.61,2024-04-28,2024-02-29


In [33]:
issues = issues[["lawyer_id","user_id","type","estimated_cost","success_rate"]]
issues.head()

,lawyer_id,user_id,type,estimated_cost,success_rate
0,12,16,Criminal,1490.15,0.11
1,14,64,Labor,2658.10,0.62
2,20,16,Civil,4313.33,0.67
3,11,96,Labor,4565.10,0.56
4,15,74,Commercial,3519.01,0.61


In [34]:
class Preprocessing_EN:
  @staticmethod
  def process(sentance):
    sentance = Preprocessing_EN.remove_punctuation(sentance)
    sentance = Preprocessing_EN.tokenizer(sentance)
    sentance = Preprocessing_EN.normalizer(sentance)
    sentance = Preprocessing_EN.remove_stopwords(sentance)
    sentance = Preprocessing_EN.remove_deplicate(sentance)
    sentance = Preprocessing_EN.stemmer(sentance)
    return sentance

  @staticmethod
  def tokenizer(sentance):
    words = word_tokenize(sentance)
    return words

  @staticmethod
  def normalizer(sentance):
    return [word.lower() for word in sentance]

  @staticmethod
  def remove_stopwords(sentance):
    stop_words = set(stopwords.words('english'))
    sentance = [word for word in sentance if word not in stop_words]
    return sentance
  
  @staticmethod
  def stemmer(sentance):
    stemmer = PorterStemmer()
    sentance = [stemmer.stem(word) for word in sentance]
    return sentance

  @staticmethod
  def remove_punctuation(sentance):
    return re.sub(r'[^A-Za-z0-9\s]',' ',sentance)

  @staticmethod
  def remove_deplicate(sentance):
    return list(set(sentance))

In [35]:
lawyers_with_rates = lawyers.merge(rates,on="lawyer_id")
lawyers_with_rates.head()

,lawyer_id,name,address,union_branch,union_number,years_of_experience,user_id,rating,review
0,107,Lawyer_107,"107 Main St, City_107",Branch_7,7277,6year,1351,1star,"Good, but communication could be improved."
1,110,Lawyer_110,"110 Main St, City_110",Branch_0,1538,9year,1358,5star,Not satisfied with the service.
2,115,Lawyer_115,"115 Main St, City_115",Branch_5,5623,7year,1515,3star,"Good, but communication could be improved."
3,121,Lawyer_121,"121 Main St, City_121",Branch_1,3844,18year,1724,4star,"Excellent service, highly recommend!"
4,122,Lawyer_122,"122 Main St, City_122",Branch_2,9335,29year,1957,4star,Very professional and helpful.


In [36]:
lawyers_with_rates["Text"] = lawyers_with_rates[['name','address','union_branch','years_of_experience','rating','review']].astype(str).agg(" ".join,axis=1)
lawyers_with_rates.head()

,lawyer_id,name,address,union_branch,union_number,years_of_experience,user_id,rating,review,Text
0,107,Lawyer_107,"107 Main St, City_107",Branch_7,7277,6year,1351,1star,"Good, but communication could be improved.","Lawyer_107 107 Main St, City_107 Branch_7 6year 1star Good, but communication could be improved."
1,110,Lawyer_110,"110 Main St, City_110",Branch_0,1538,9year,1358,5star,Not satisfied with the service.,"Lawyer_110 110 Main St, City_110 Branch_0 9year 5star Not satisfied with the service."
2,115,Lawyer_115,"115 Main St, City_115",Branch_5,5623,7year,1515,3star,"Good, but communication could be improved.","Lawyer_115 115 Main St, City_115 Branch_5 7year 3star Good, but communication could be improved."
3,121,Lawyer_121,"121 Main St, City_121",Branch_1,3844,18year,1724,4star,"Excellent service, highly recommend!","Lawyer_121 121 Main St, City_121 Branch_1 18year 4star Excellent service, highly recommend!"
4,122,Lawyer_122,"122 Main St, City_122",Branch_2,9335,29year,1957,4star,Very professional and helpful.,"Lawyer_122 122 Main St, City_122 Branch_2 29year 4star Very professional and helpful."


In [37]:
lawyers_with_rates['processed_text'] = lawyers_with_rates['Text'].apply(Preprocessing_EN.process)
lawyers_with_rates.head()

,lawyer_id,name,address,union_branch,union_number,years_of_experience,user_id,rating,review,Text,processed_text
0,107,Lawyer_107,"107 Main St, City_107",Branch_7,7277,6year,1351,1star,"Good, but communication could be improved.","Lawyer_107 107 Main St, City_107 Branch_7 6year 1star Good, but communication could be improved.","[7, 1star, branch, 107, citi, improv, commun, lawyer, good, st, 6year, could, main]"
1,110,Lawyer_110,"110 Main St, City_110",Branch_0,1538,9year,1358,5star,Not satisfied with the service.,"Lawyer_110 110 Main St, City_110 Branch_0 9year 5star Not satisfied with the service.","[branch, satisfi, citi, 110, lawyer, st, 0, servic, 9year, 5star, main]"
2,115,Lawyer_115,"115 Main St, City_115",Branch_5,5623,7year,1515,3star,"Good, but communication could be improved.","Lawyer_115 115 Main St, City_115 Branch_5 7year 3star Good, but communication could be improved.","[7year, branch, citi, 5, improv, commun, 3star, lawyer, good, st, 115, could, main]"
3,121,Lawyer_121,"121 Main St, City_121",Branch_1,3844,18year,1724,4star,"Excellent service, highly recommend!","Lawyer_121 121 Main St, City_121 Branch_1 18year 4star Excellent service, highly recommend!","[excel, branch, 1, citi, lawyer, recommend, st, servic, highli, 18year, 4star, 121, main]"
4,122,Lawyer_122,"122 Main St, City_122",Branch_2,9335,29year,1957,4star,Very professional and helpful.,"Lawyer_122 122 Main St, City_122 Branch_2 29year 4star Very professional and helpful.","[help, 122, branch, profession, citi, lawyer, st, 2, 29year, 4star, main]"


In [38]:
documents = lawyers_with_rates[['lawyer_id','processed_text']].copy()
documents['processed_text'] = documents['processed_text'].apply(lambda x:" ".join(x))
documents.rename(columns={
    "lawyer_id":"docno",
    "processed_text":"text"
},inplace=True)
documents.head()

,docno,text
0,107,7 1star branch 107 citi improv commun lawyer good st 6year could main
1,110,branch satisfi citi 110 lawyer st 0 servic 9year 5star main
2,115,7year branch citi 5 improv commun 3star lawyer good st 115 could main
3,121,excel branch 1 citi lawyer recommend st servic highli 18year 4star 121 main
4,122,help 122 branch profession citi lawyer st 2 29year 4star main


In [39]:
index_path = r'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index'
indexer = pt.DFIndexer(index_path,overwrite=True)

index_ref = indexer.index(documents['text'],documents['docno'].astype(str))

print("Index created with reference:", index_ref)

C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_13016\485533886.py:2: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer(index_path,overwrite=True)


19:58:49.564 [main] ERROR org.terrier.structures.indexing.Indexer -- Could not rename index
java.io.IOException: Rename of index structure file 'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index/data_1.direct.bf' (exists) to 'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:355)
Index created with reference: <org.terrier.querying.IndexRef at 0x14bde1a5fe0 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x-8b167a6 at 0x14b97463570>>


In [40]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Number of documents: 20
Number of terms: 55
Number of postings: 208
Number of fields: 0
Number of tokens: 208
Field names: []
Positions:   false



In [41]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
query = "experienced lawyer"
results = bm25.search(query)
print(results[['docno', 'score']])

   docno     score
0    121 -5.040329
1    144 -5.040329
2    164 -5.040329
3    146 -5.234022
4    169 -5.234022
5    107 -5.443197
6    110 -5.443197
7    115 -5.443197
8    122 -5.443197
9    127 -5.443197
10   127 -5.443197
11   134 -5.443197
12   141 -5.443197
13   148 -5.443197
14   175 -5.443197
15   180 -5.443197
16   181 -5.443197
17   189 -5.443197
18   190 -5.443197
19   197 -5.443197


C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_13016\2634325464.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

In [43]:
user_input = "experienced lawyer with 3years"

In [54]:
all_descriptions = [user_input] + documents["text"].tolist()
all_descriptions

['experienced lawyer with 3years',
 '7 1star branch 107 citi improv commun lawyer good st 6year could main',
 'branch satisfi citi 110 lawyer st 0 servic 9year 5star main',
 '7year branch citi 5 improv commun 3star lawyer good st 115 could main',
 'excel branch 1 citi lawyer recommend st servic highli 18year 4star 121 main',
 'help 122 branch profession citi lawyer st 2 29year 4star main',
 '7 4year branch 2star profession citi help lawyer st 127 main',
 '7 4year branch citi improv commun lawyer good st could 127 5star main',
 'branch 134 citi 4 improv commun 3star lawyer good st 13year could main',
 'branch satisfi 1 citi lawyer st 30year servic 141 5star main',
 'excel 1star branch citi 4 144 lawyer 3year recommend st servic highli main',
 '4year branch would citi experi lawyer 6 st hire except 4star 146 main',
 'help branch profession citi 8 16year 3star lawyer st 148 main',
 'excel 1star branch citi 4 16year lawyer recommend st servic highli 164 main',
 '9 branch 1star would citi e

In [55]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(all_descriptions)

In [56]:
user_vector = tfidf_matrix[0]  
lawyer_vectors = tfidf_matrix[1:]

In [57]:
similarities = cosine_similarity(user_vector, lawyer_vectors).flatten()

In [58]:
ranked_lawyers = np.argsort(similarities)[::-1]

In [59]:
print("\nTop Recommendations:")
for idx in ranked_lawyers:
    lawyer_id = documents.iloc[idx]["docno"]
    description = documents.iloc[idx]["text"]
    similarity = similarities[idx]
    print(f"Lawyer ID: {lawyer_id}, Similarity: {similarity:.4f}, Description: {description}")


Top Recommendations:
Lawyer ID: 180, Similarity: 0.0329, Description: branch satisfi citi lawyer st 0 servic 18year 4star 180 main
Lawyer ID: 127, Similarity: 0.0324, Description: 7 4year branch 2star profession citi help lawyer st 127 main
Lawyer ID: 189, Similarity: 0.0322, Description: 9 branch 189 satisfi citi lawyer 3year st servic 4star main
Lawyer ID: 110, Similarity: 0.0319, Description: branch satisfi citi 110 lawyer st 0 servic 9year 5star main
Lawyer ID: 190, Similarity: 0.0309, Description: branch satisfi citi lawyer 10year st 0 servic 190 5star main
Lawyer ID: 141, Similarity: 0.0309, Description: branch satisfi 1 citi lawyer st 30year servic 141 5star main
Lawyer ID: 127, Similarity: 0.0308, Description: 7 4year branch citi improv commun lawyer good st could 127 5star main
Lawyer ID: 148, Similarity: 0.0308, Description: help branch profession citi 8 16year 3star lawyer st 148 main
Lawyer ID: 122, Similarity: 0.0306, Description: help 122 branch profession citi lawyer st

In [ ]:
recommendations = lawyers.iloc[ranked_lawyers].copy()
recommendations["similarity"] = similarities[ranked_lawyers]
recommendations.to_csv("user_recommendations.csv", index=False)